In [1]:
import numpy as np
import scipy.io as scio
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch
import random
import os
from torch.autograd import Variable
from torch.nn import Linear,Tanh,Sequential
import torch.nn as nn
import math
import scipy.io as scio
import time
from tqdm import tqdm
import pickle
import heapq

In [2]:
class Rational(torch.nn.Module):
    def __init__(self,
                 Data_Type = torch.float32,
                 Device    = torch.device('cpu')):
        # This activation function is based on the following paper:
        # Boulle, Nicolas, Yuji Nakatsukasa, and Alex Townsend. "Rational neural
        # networks." arXiv preprint arXiv:2004.01902 (2020).

        super(Rational, self).__init__()

        # Initialize numerator and denominator coefficients to the best
        # rational function approximation to ReLU. These coefficients are listed
        # in appendix A of the paper.
        self.a = torch.nn.parameter.Parameter(
                        torch.tensor((1.1915, 1.5957, 0.5, .0218),
                                     dtype = Data_Type,
                                     device = Device))
        self.a.requires_grad_(True)

        self.b = torch.nn.parameter.Parameter(
                        torch.tensor((2.3830, 0.0, 1.0),
                                     dtype = Data_Type,
                                     device = Device))
        self.b.requires_grad_(True)

    def forward(self, X : torch.tensor):
        """ This function applies a rational function to each element of X.
        ------------------------------------------------------------------------
        Arguments:
        X: A tensor. We apply the rational function to every element of X.
        ------------------------------------------------------------------------
        Returns:
        Let N(x) = sum_{i = 0}^{3} a_i x^i and D(x) = sum_{i = 0}^{2} b_i x^i.
        Let R = N/D (ignoring points where D(x) = 0). This function applies R
        to each element of X and returns the resulting tensor. """

        # Create aliases for self.a and self.b. This makes the code cleaner.
        a = self.a
        b = self.b

        # Evaluate the numerator and denominator. Because of how the * and +
        # operators work, this gets applied element-wise.
        N_X = a[0] + X*(a[1] + X*(a[2] + a[3]*X))
        D_X = b[0] + X*(b[1] + b[2]*X)

        # Return R = N_X/D_X. This is also applied element-wise.
        return N_X/D_X

class Sin(nn.Module):
    def __init__(self):
        super(Sin, self).__init__()

    def forward(self, x):
        x = torch.sin(x)
        return x
        
class NN(torch.nn.Module):
    def __init__(self,
                 Num_Hidden_Layers   : int          = 3,
                 Neurons_Per_Layer   : int          = 20,   # Neurons in each Hidden Layer
                 Input_Dim           : int          = 1,    # Dimension of the input
                 Output_Dim          : int          = 1,    # Dimension of the output
                 Data_Type           : torch.dtype  = torch.float32,
                 Device              : torch.device = torch.device('cpu'),
                 Activation_Function : str          = "Tanh",
                 Batch_Norm          : bool         = False):
        # For the code below to work, Num_Hidden_Layers, Neurons_Per_Layer,
        # Input_Dim, and Output_Dim must be positive integers.
        assert(Num_Hidden_Layers   > 0), "Num_Hidden_Layers must be positive. Got %du" % Num_Hidden_Layers;
        assert(Neurons_Per_Layer   > 0), "Neurons_Per_Layer must be positive. Got %u" % Neurons_Per_Layer;
        assert(Input_Dim           > 0), "Input_Dim must be positive. Got %u"  % Input_Dim;
        assert(Output_Dim          > 0), "Output_Dim must be positive. Got %u" % Output_Dim;

        super(NN, self).__init__()

        # Define object attributes.
        self.Input_Dim          : int  = Input_Dim
        self.Output_Dim         : int  = Output_Dim
        self.Num_Hidden_Layers  : int  = Num_Hidden_Layers
        self.Batch_Norm         : bool = Batch_Norm

        # Initialize the Layers. We hold all layers in a ModuleList.
        self.Layers = torch.nn.ModuleList()

        # Initialize Batch Normalization, if we're doing that.
        if(Batch_Norm == True):
            self.Norm_Layer = torch.nn.BatchNorm1d(
                                    num_features = Input_Dim,
                                    dtype        = Data_Type,
                                    device       = Device)

        # Append the first hidden layer. The domain of this layer is
        # R^{Input_Dim}. Thus, in_features = Input_Dim. Since this is a hidden
        # layer, its co-domain is R^{Neurons_Per_Layer}. Thus, out_features =
        # Neurons_Per_Layer.
        self.Layers.append(torch.nn.Linear(
                                in_features  = Input_Dim,
                                out_features = Neurons_Per_Layer,
                                bias         = True ).to(dtype = Data_Type, device = Device))

        # Now append the rest of the hidden layers. Each maps from
        # R^{Neurons_Per_Layer} to itself. Thus, in_features = out_features =
        # Neurons_Per_Layer. We start at i = 1 because we already created the
        # 1st hidden layer.
        for i in range(1, Num_Hidden_Layers):
            self.Layers.append(torch.nn.Linear(
                                    in_features  = Neurons_Per_Layer,
                                    out_features = Neurons_Per_Layer,
                                    bias         = True ).to(dtype = Data_Type, device = Device))

        # Now, append the Output Layer, which has Neurons_Per_Layer input
        # features, but only Output_Dim output features.
        self.Layers.append(torch.nn.Linear(
                                in_features  = Neurons_Per_Layer,
                                out_features = Output_Dim,
                                bias         = True ).to(dtype = Data_Type, device = Device))

        # Initialize the weight matrices, bias vectors in the network.
        if(Activation_Function == "Tanh" or Activation_Function == "Rational"):
            Gain : float = 0
            if  (Activation_Function == "Tanh"):
                Gain = 5./3.
            elif(Activation_Function == "Rational"):
                Gain = 1.41

            for i in range(self.Num_Hidden_Layers + 1):
                torch.nn.init.xavier_normal_(self.Layers[i].weight, gain = Gain)
                torch.nn.init.zeros_(self.Layers[i].bias)

        elif(Activation_Function == "Sin"):
            # The SIREN paper suggests initializing the elements of every weight
            # matrix (except for the first one) by sampling a uniform
            # distribution over [-c/root(n), c/root(n)], where c > root(6),
            # and n is the number of neurons in the layer. I use c = 3 > root(6).
            #
            # Further, for simplicity, I initialize each bias vector to be zero.
            a : float = 3./math.sqrt(Neurons_Per_Layer)
            for i in range(0, self.Num_Hidden_Layers + 1):
                torch.nn.init.uniform_( self.Layers[i].weight, -a, a)
                torch.nn.init.zeros_(   self.Layers[i].bias)

        # Finally, set the Network's activation functions.
        self.Activation_Functions = torch.nn.ModuleList()
        if  (Activation_Function == "Tanh"):
            for i in range(Num_Hidden_Layers):
                self.Activation_Functions.append(torch.nn.Tanh())
        elif(Activation_Function == "Sin"):
            for i in range(Num_Hidden_Layers):
                self.Activation_Functions.append(Sin())
        elif(Activation_Function == "Rational"):
            for i in range(Num_Hidden_Layers):
                self.Activation_Functions.append(Rational(Data_Type = Data_Type, Device = Device))
        else:
            print("Unknown Activation Function. Got %s" % Activation_Function)
            print("Thrown by Neural_Network.__init__. Aborting.")
            exit();



    def forward(self, X : torch.Tensor) -> torch.Tensor:
        """ Forward method for the NN class. Note that the user should NOT call
        this function directly. Rather, they should call it through the __call__
        method (using the NN object like a function), which is part of the
        module class and calls forward.

        ------------------------------------------------------------------------
        Arguments:

        X: A batch of inputs. This should be a B by Input_Dim tensor, where B
        is the batch size. The ith row of X should hold the ith input.

        ------------------------------------------------------------------------
        Returns:

        If X is a B by Input_Dim tensor, then the output of this function is a
        B by Output_Dim tensor, whose ith row holds the value of the network
        applied to the ith row of X. """

        # If we are using batch normalization, then normalize the inputs.
        if(self.Batch_Norm == True):
            X = self.Norm_Layer(X);

        # Pass X through the hidden layers. Each has an activation function.
        for i in range(0, self.Num_Hidden_Layers):
            X = self.Activation_Functions[i](self.Layers[i](X));

        # Pass through the last layer (with no activation function) and return.
        return self.Layers[self.Num_Hidden_Layers](X);

In [3]:
def random_data(total, choose,choose_validate,x,t,un,x_num,t_num,random_seed=525):
    random.seed(random_seed)
    data=np.zeros(2)
    h_data=np.zeros([total,1])
    database=np.zeros([total,2])
    num=0


    for j in range(x_num):
        for i in range(t_num):
            data[0]=x[j]
            data[1]=t[i]
            h_data[num]=un[j,i]
            database[num]=data
            num+=1
    state = np.random.get_state()
    np.random.shuffle(database)
    np.random.set_state(state)
    np.random.shuffle(h_data)
    h_data_choose = h_data[0:choose]
    database_choose = database[0:choose]
    h_data_validate = h_data[choose:choose + choose_validate]
    database_validate = database[choose:choose + choose_validate]

    h_data_choose = torch.from_numpy(h_data_choose.astype(np.float32))
    database_choose = torch.from_numpy(database_choose.astype(np.float32))
    h_data_validate = torch.from_numpy(h_data_validate.astype(np.float32))
    database_validate = torch.from_numpy(database_validate.astype(np.float32))
   
    return h_data_choose,h_data_validate,database_choose,database_validate

In [4]:
def Generate_meta_data(Net,Equation_name, choose, noise_level, Load_state, x_low, x_up, t_low, t_up, nx=100,
                       nt=100, ):
    Net.load_state_dict(torch.load(f'model_save/{Equation_name}/{choose}_{noise_level}/{Load_state}.pkl'))
    Net.eval()

    x = torch.linspace(x_low, x_up, nx)
    t = torch.linspace(t_low, t_up, nt)
    total = nx * nt

    num = 0
    data = torch.zeros(2)
    h_data = torch.zeros([total, 1])
    database = torch.zeros([total, 2])
    for j in range(nx):
        for i in range(nt):
            data[0] = x[j]
            data[1] = t[i]
            database[num] = data
            num += 1

    database = Variable(database, requires_grad=True).to(device)
    PINNstatic=Net(database)

    H_grad = torch.autograd.grad(outputs=PINNstatic.sum(), inputs=database, create_graph=True)[0]
    Hx=H_grad[:,0].reshape(total,1)
    Ht=H_grad[:,1].reshape(total,1)
    Hxx=torch.autograd.grad(outputs=Hx.sum(), inputs=database,create_graph=True)[0][:,0].reshape(total,1)
    Hxxx=torch.autograd.grad(outputs=Hxx.sum(), inputs=database,create_graph=True)[0][:,0].reshape(total,1)
    Htt=torch.autograd.grad(outputs=Ht.sum(), inputs=database,create_graph=True)[0][:,1].reshape(total,1)
    Theta=torch.concatenate([PINNstatic,Hx,Hxx,Hxxx,Ht,Htt],axis=1) 
 
    return Theta.cpu().data.numpy(),x.cpu().data.numpy(),t.cpu().data.numpy()

In [5]:
class GA():
    def __init__(self,x,t,epi,R):
        self.max_length = 5
        self.partial_prob = 0.6
        self.genes_prob = 0.6
        self.mutate_rate = 0.4
        self.delete_rate = 0.5
        self.add_rate = 0.4
        self.pop_size = 400
        self.n_generations = 100
        self.u=R[:,0].reshape(R.shape[0],1)
        self.u_x=R[:,1].reshape(R.shape[0],1)
        self.u_xx=R[:,2].reshape(R.shape[0],1)
        self.u_xxx=R[:,3].reshape(R.shape[0],1)
        self.u_t=R[:,4].reshape(R.shape[0],1)
        self.u_tt=R[:,5].reshape(R.shape[0],1)
        self.x=x
        self.t=t
        self.dx=x[1]-x[0]
        self.dt=t[1]-t[0]
        self.nx=x.shape[0]
        self.nt=t.shape[0]
        self.total=self.nx*self.nt
        self.epi=epi



    def random_module(self):
        genes_module=[]
        for i in range(self.max_length):
            a = random.randint(0, 3)
            genes_module.append(a)
            prob=random.uniform(0,1)
            if prob>self.partial_prob:
                break
        return genes_module

    def random_genome(self):
        genes=[]

        for i in range(self.max_length):
            gene_random=GA.random_module(self)
            genes.append(sorted(gene_random))
            prob=random.uniform(0,1)
            if prob>self.genes_prob:
                break
        return genes

    def translate_DNA(self,gene):
        gene_translate=np.ones([self.total,1])
        length_penalty_coef=0
        for k in range(len(gene)):
            gene_module=gene[k]
            length_penalty_coef+=len(gene_module)
            module_out=np.ones([self.u.shape[0],self.u.shape[1]])
            for i in gene_module:
                if i==0:
                    temp=self.u
                if i==1:
                    temp=self.u_x
                if i==2:
                    temp=self.u_xx
                if i==3:
                    temp=self.u_xxx
                module_out*=temp
            gene_translate=np.hstack((gene_translate,module_out))
        gene_translate=np.delete(gene_translate,[0],axis=1)
        return gene_translate,length_penalty_coef

    def get_fitness(self,gene_translate,length_penalty_coef):
        u_t=self.u_t
        u, d, v = np.linalg.svd(np.hstack((u_t, gene_translate)), full_matrices=False)
        coef_NN = v.T[:, -1] / (v.T[:, -1][0] + 1e-8)
        coef=-coef_NN[1:].reshape(coef_NN.shape[0]-1,1)
        res = u_t-np.dot(gene_translate,coef)

        u_tt=self.u_tt
        u, d, v = np.linalg.svd(np.hstack((u_tt, gene_translate)), full_matrices=False)
        coef_NN = v.T[:, -1] / (v.T[:, -1][0] + 1e-8)
        coef_tt=-coef_NN[1:].reshape(coef_NN.shape[0]-1,1)
        res_tt = u_tt-np.dot(gene_translate,coef_tt)

        MSE_true = np.sum(np.array(res) ** 2) / self.total
        MSE_true_tt = np.sum(np.array(res_tt) ** 2) / self.total
        
        if MSE_true<MSE_true_tt:
            name='u_t'
            MSE=MSE_true+self.epi*length_penalty_coef
            coef=coef
            return coef,MSE,MSE_true,name


        if MSE_true>MSE_true_tt:
            name='u_tt'
            MSE = MSE_true_tt + self.epi * length_penalty_coef
            return coef_tt, MSE, MSE_true_tt, name




    def cross_over(self):
        Chrom, size_pop = self.Chrom,self.n_generations
        Chrom1, Chrom2 = Chrom[::2], Chrom[1::2]
        for i in range(int(size_pop / 2)):
            n1= np.random.randint(0, len(Chrom1[i]))
            n2=np.random.randint(0, len(Chrom2[i]))

            father=Chrom1[i][n1].copy()
            mother=Chrom2[i][n2].copy()

            Chrom1[i][n1]=mother
            Chrom2[i][n2]=father

        Chrom[::2], Chrom[1::2] = Chrom1, Chrom2
        self.Chrom = Chrom
        return self.Chrom

    def mutation(self):
        Chrom, size_pop = self.Chrom, self.pop_size

        for i in range(size_pop):
            n1 = np.random.randint(0, len(Chrom[i]))

            # ------------add module---------------
            prob = np.random.uniform(0, 1)
            if prob < self.add_rate:
                add_Chrom = GA.random_module(self)
                if add_Chrom not in Chrom[i]:
                    Chrom[i].append(add_Chrom)

            # --------delete module----------------
            prob = np.random.uniform(0, 1)
            if prob < self.delete_rate:
                if len(Chrom[i]) > 1:
                    delete_index = np.random.randint(0, len(Chrom[i]))
                    Chrom[i].pop(delete_index)

            # ------------gene mutation------------------
            prob = np.random.uniform(0, 1)
            if prob < self.mutate_rate:
                if len(Chrom[i]) > 0:
                    n1 = np.random.randint(0, len(Chrom[i]))
                    n2 = np.random.randint(0, len(Chrom[i][n1]))
                    Chrom[i][n1][n2] = random.randint(0,3)
        self.Chrom = Chrom
        return self.Chrom


    def select(self):  # nature selection wrt pop's fitness
        Chrom, size_pop = self.Chrom, self.pop_size
        new_Chrom=[]
        new_fitness=[]
        new_coef=[]
        new_name=[]

        fitness_list = []
        coef_list=[]
        name_list=[]

        for i in range(size_pop):
            gene_translate, length_penalty_coef = GA.translate_DNA(self, Chrom[i])
            coef, MSE, MSE_true,name = GA.get_fitness(self, gene_translate, length_penalty_coef)
            fitness_list.append(MSE)
            coef_list.append(coef)
            name_list.append(name)
        re1 = list(map(fitness_list.index, heapq.nsmallest(int(size_pop/2), fitness_list)))

        for index in re1:
            new_Chrom.append(Chrom[index])
            new_fitness.append(fitness_list[index])
            new_coef.append(coef_list[index])
            new_name.append(name_list[index])
        for index in range(int(size_pop/2)):
            new=GA.random_genome(self)
            new_Chrom.append(new)


        self.Chrom=new_Chrom
        self.Fitness=new_fitness
        self.coef=new_coef
        self.name=new_name
        return self.Chrom,self.Fitness,self.coef,self.name

    def delete_duplicates(self):
        Chrom, size_pop = self.Chrom,self.pop_size
        for i in range(size_pop):
            new_genome=[]
            for j in range(len(Chrom[i])):
                if sorted(Chrom[i][j]) not in new_genome:
                    new_genome.append(sorted(Chrom[i][j]))
            Chrom[i]=new_genome
        self.Chrom=Chrom
        return self.Chrom
    
    def convert_chrom_to_eq(self,chrom,left_name,coef):
        name=['u','ux','uxx','uxxx','ut','utt']
        string=[]
        for i in range(len(chrom)):
            item=chrom[i]
            string.append(str(np.round(coef[i,0],4)))
            string.append('*')
            for gene in item:
                string.append(name[gene])
                string.append('*')
            string.pop(-1)
            string.append('+')
        string.pop(-1)
        string=f"{left_name}="+''.join(string)
        return string
            
                

    def evolution(self):
        self.Chrom = []
        self.Fitness=[]
        for iter in range(self.pop_size):
            intial_genome =GA.random_genome(self)
            self.Chrom.append(intial_genome)
            gene_translate, length_penalty_coef=GA.translate_DNA(self,intial_genome)
            coef, MSE,MSE_true,name=GA.get_fitness(self,gene_translate,length_penalty_coef)
            self.Fitness.append(MSE)
        GA.delete_duplicates(self)
        try:
            os.makedirs(f'result_save/{Equation_name}/{choose}_{noise_level}')
        except OSError:
            pass

        with open(f'result_save/{Equation_name}/{choose}_{noise_level}/DLGA_output.txt', "a") as f:
            date_str = time.strftime('%Y-%m-%d  %H:%M:%S', time.localtime())
            f.write(f'\n============Run at {date_str}==============\n')
            f.write(f'============Params=============\n')
            f.write(f'#l0_penalty:{self.epi}\n')
            f.write(f'#pop_size:{self.pop_size}\n')
            f.write(f'#generations:{self.n_generations}\n')
            f.write(f'============results=============\n')
        for iter in tqdm(range(self.n_generations)):
            pickle.dump(self.Chrom.copy()[0],open(f'result_save/{Equation_name}/{choose}_{noise_level}/best_save.pkl','wb'))
            best =self.Chrom.copy()[0]
            GA.cross_over(self)
            GA.mutation(self)
            GA.delete_duplicates(self)
            best = pickle.load(open(f'result_save/{Equation_name}/{choose}_{noise_level}/best_save.pkl','rb'))
            self.Chrom[0]=best
            GA.select(self)
            if self.Chrom[0]!=best:
                with open(f'result_save/{Equation_name}/{choose}_{noise_level}/DLGA_output.txt', "a") as f:
                    f.write(f'iter: {iter+1}\n')
                    f.write(f'The best Chrom: {self.Chrom[0]}\n')
                    f.write(f'The best coef:  \n{self.coef[0]}\n')
                    f.write(f'The best fitness: {self.Fitness[0]}\n')
                    f.write(f'The best name: {self.name[0]}\n')
                    f.write(f'----------------------------------------\n')
                    print(f'iter: {iter+1}\n')
                    print(f'The best Chrom: {self.Chrom[0]}')
                    print(f'The best coef:  \n{self.coef[0]}')
                    print(f'The best fitness: {self.Fitness[0]}')
                    print(f'The best name: {self.name[0]}\r')
        print('-------------------------------------------')
        print(f'Finally discovered equation')
        print(f'The best Chrom: {self.Chrom[0]}')
        print(f'The best coef:  \n{self.coef[0]}')
        print(f'The best fitness: {self.Fitness[0]}')
        print(f'The best name: {self.name[0]}\r')
        print('---------------------------------------------')

        return self.Chrom[0],self.coef[0],self.Fitness[0],self.name[0]

In [6]:
Equation_name='KdV_equation'
choose=10000
noise_level=0
if Equation_name=='KdV_equation':
    data_path = fr"KdV_equation.mat"
    data = scio.loadmat(data_path)
    un = data.get("uu")
    x = np.squeeze(data.get("x"))
    t = np.squeeze(data.get("tt").reshape(1, 201))
    x_low = -0.8
    x_up = 0.8
    t_low = 0.1
    t_up = 0.9
    target = [[3], [0, 1]]
    Left = 'u_t'
    epi = 1e-1
    Delete_equation_name='KdV equation'
x_num=x.shape[0]
t_num=t.shape[0]
total=x_num*t_num
choose_validate=5000
meta_data_num=10000
print(f"dataset size: x_num {x_num}  t_num  {t_num}")

dataset size: x_num 512  t_num  201


In [7]:
# add noise
noise_value=(noise_level/100)*np.std(un)*np.random.randn(*un.shape)
un=un+noise_value
#save model dir

try:
    os.makedirs(f'noise_data_save/{Equation_name}/')
    np.save(f'noise_data_save/{Equation_name}/un_{choose}_{noise_level}.npy', un)
except OSError:
    un=np.load(f'noise_data_save/{Equation_name}/un_{choose}_{noise_level}.npy')
    print('===load noisy data===')
    pass

===load noisy data===


In [8]:
torch.manual_seed(525)
torch.cuda.manual_seed(525)
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
Net=NN(Num_Hidden_Layers=5,
    Neurons_Per_Layer=50,
    Input_Dim=2,
    Output_Dim=1,
    Data_Type=torch.float32,
    Device='cuda',
    Activation_Function="Sin",
    Batch_Norm=False)
try:
    os.makedirs(f'model_save/{Equation_name}/{choose}_{noise_level}')
except OSError:
    pass

#=========produce random dataset==========
h_data_choose,h_data_validate,database_choose,database_validate=random_data(total,choose,choose_validate,x,t,un,x_num,t_num)
database_choose = Variable(database_choose.cuda(),requires_grad=True)
database_validate = Variable(database_validate.cuda(),requires_grad=True)
h_data_choose=Variable(h_data_choose.cuda())
h_data_validate=Variable(h_data_validate.cuda())

print("Training data:",database_choose.shape,"Validating data",database_validate.shape)

Training data: torch.Size([10000, 2]) Validating data torch.Size([5000, 2])


In [9]:
NN_optimizer = torch.optim.Adam([
        {'params': Net.parameters()},
    ])


MSELoss = torch.nn.MSELoss()
validate_error=[]
best_validate_error = []
loss_back = 1e8
flag = 0
print(f'===============train Net=================')
file = open(f'model_save/{Equation_name}/{choose}_{noise_level}/loss.txt', 'w').close()
file=open(f'model_save/{Equation_name}/{choose}_{noise_level}/loss.txt',"a+")
for iter in range(50000):
    NN_optimizer.zero_grad()
    prediction = Net(database_choose)
    prediction_validate = Net(database_validate).cpu().data.numpy()
    loss = MSELoss(h_data_choose, prediction)
    loss_validate = np.sum((h_data_validate.cpu().data.numpy() - prediction_validate) ** 2) / choose_validate
    loss.backward()
    NN_optimizer.step()



    if (iter+1) % 500 == 0:
        validate_error.append(loss_validate)
        torch.save(Net.state_dict(),
                   f'model_save/{Equation_name}/{choose}_{noise_level}/' + f"Net_{iter + 1}.pkl")

        print("iter_num: %d      loss: %.8f    loss_validate: %.8f" % (iter+1, loss, loss_validate))
        file.write("iter_num: %d      loss: %.8f    loss_validate: %.8f \n" % (iter+1, loss, loss_validate))
file.close()
best_epoch=(validate_error.index(min(validate_error))+1)*500
print(best_epoch)
np.save(f'model_save/{Equation_name}/{choose}_{noise_level}/best_epoch.npy',np.array([best_epoch]))


===============train Net=================
iter_num: 500      loss: 0.00218572    loss_validate: 0.00240086
iter_num: 1000      loss: 0.00048595    loss_validate: 0.00052037
iter_num: 1500      loss: 0.00027203    loss_validate: 0.00028582
iter_num: 2000      loss: 0.00020035    loss_validate: 0.00020832
iter_num: 2500      loss: 0.00017430    loss_validate: 0.00018839
iter_num: 3000      loss: 0.00013247    loss_validate: 0.00014521
iter_num: 3500      loss: 0.00011370    loss_validate: 0.00012781
iter_num: 4000      loss: 0.00009687    loss_validate: 0.00011427
iter_num: 4500      loss: 0.00008539    loss_validate: 0.00010617
iter_num: 5000      loss: 0.00015732    loss_validate: 0.00018393
iter_num: 5500      loss: 0.00006567    loss_validate: 0.00009272
iter_num: 6000      loss: 0.00005800    loss_validate: 0.00008577
iter_num: 6500      loss: 0.00023031    loss_validate: 0.00024528
iter_num: 7000      loss: 0.00004532    loss_validate: 0.00006954
iter_num: 7500      loss: 0.0000392

In [10]:
best_epoch=np.load(f'model_save/{Equation_name}/{choose}_{noise_level}/best_epoch.npy')[0]
print("best_epoch:",best_epoch)
Load_state = 'Net' +  f'_{best_epoch}'
Theta,x_meta,t_meta=Generate_meta_data(Net,Equation_name,choose,noise_level,Load_state,x_low,x_up,t_low,t_up)
print(Theta.shape)

best_epoch: 48500
(10000, 6)


In [11]:
opt_GA=GA(x_meta,t_meta,epi,Theta)
Chrom,coef,_,name=opt_GA.evolution()
print("equation form:",opt_GA.convert_chrom_to_eq(Chrom,name,coef))

  1%|▊                                                                                 | 1/100 [00:00<01:05,  1.50it/s]

iter: 1

The best Chrom: [[3], [0, 1]]
The best coef:  
[[-0.00248602]
 [-0.99451355]]
The best fitness: 0.33101108574304633
The best name: u_t


  5%|████                                                                              | 5/100 [00:02<00:53,  1.77it/s]

iter: 5

The best Chrom: [[0, 1], [3]]
The best coef:  
[[-0.99451355]
 [-0.00248602]]
The best fitness: 0.33101108574304505
The best name: u_t


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [01:02<00:00,  1.60it/s]

-------------------------------------------
Finally discovered equation
The best Chrom: [[0, 1], [3]]
The best coef:  
[[-0.99451355]
 [-0.00248602]]
The best fitness: 0.33101108574304505
The best name: u_t
---------------------------------------------
equation form: u_t=-0.9945*u*ux+-0.0025*uxxx
